In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Set path to line catalogue
import os
if "ARTS_DATA_PATH" not in os.environ:
    os.environ["ARTS_DATA_PATH"] = "/work/um0878/users/olemke/rtcourse/arts-xml-data/"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import typhon as ty
from pyarts import xml
from olr_module import calc_olr

plt.style.use(ty.plots.styles("typhon"))

In [ ]:
# Read input atmosphere
atmfield = xml.load("input/midlatitude-summer.xml")

# Scale the CO2 concentration
atmfield.scale("abs_species-CO2", 1)
# atmfield.add("T", 0)
atmfield.set("T", atmfield.get("T") + 50)

# Calculate the outgoing-longwave radiation
f, olr = calc_olr(atmfield, verbosity=0)

In [ ]:
# Plotting.
import matplotlib
font = {'size'   : 24}
matplotlib.rc('font', **font)

wn = ty.physics.frequency2wavenumber(f) / 100  # Hz -> cm^-1

temps = [225, 250, 275, atmfield.get("T", keep_dims=False)[0]]
temp_colors = ty.plots.cmap2rgba("temperature", len(temps))

fig, ax = plt.subplots()
for t, color in sorted(zip(temps, temp_colors)):
    ax.plot(
        wn, np.pi * ty.physics.planck(f, t), label=f"{t:3.1f} K", color=color
    )
ax.plot(wn, olr, color="C0", label="Radiance")
ax.legend()
ax.set_title(rf"OLR={np.trapz(olr, f):3.2f} $\sf Wm^{{-2}}$")
ax.set_xlim(wn.min(), wn.max())
ax.set_xlabel(r"Wavenumber [$\sf cm^{-1}$]")
ax.set_ylabel(r"Irradiance [$\sf Wm^{-2}Hz^{-1}$]")
ax.set_ylim(bottom=0)
fig.savefig("plots/olr.pdf")